In [1]:
# Install required libraries
!pip install pinecone sentence-transformers

from sentence_transformers import SentenceTransformer
import pinecone

In [4]:
# ---------- 1️⃣ Initialize Pinecone ----------
# Replace with your actual API key from https://app.pinecone.io
from pinecone import Pinecone, ServerlessSpec

# Make sure to replace "YOUR_API_KEY" with your actual API key
pc = Pinecone(api_key="pcsk_GM9g5_QN7xXLte8WgChVN4XU7H5VeAiaTxKMRQzktRjvqS3wW8q8adnvVv7ety81tZcCN")

# Create or connect to an index (dimension must match your embedding size)
index_name = "sentence-similarity"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',  # Choose your preferred cloud provider
            region='us-east-1'  # Choose your preferred region
        )
    )

index = pc.Index(index_name)

In [5]:
# ---------- 2️⃣ Prepare data ----------
sentences = [
    "The sun rises in the east.",
    "I love playing football with my friends.",
    "Artificial intelligence is transforming industries.",
    "The cat is sleeping on the couch.",
    "Python is a popular programming language.",
    "He went to the store to buy some milk.",
    "Machine learning models learn from data.",
    "The weather today is very pleasant.",
    "She enjoys reading books in her free time.",
    "New York City is known as the city that never sleeps."
]

# ---------- 3️⃣ Generate embeddings ----------
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(sentences)

# ---------- 4️⃣ Upsert data into Pinecone ----------
vectors = [
    (f"id{i}", embeddings[i].tolist(), {"sentence": sentences[i]})
    for i in range(len(sentences))
]
index.upsert(vectors)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'upserted_count': 10}

In [6]:
# ---------- 5️⃣ Query with a new sentence ----------
query_sentence = "AI is changing how companies work."
query_embedding = model.encode([query_sentence]).tolist()

# ---------- 6️⃣ Search in Pinecone ----------
results = index.query(vector=query_embedding[0], top_k=1, include_metadata=True)

# ---------- 7️⃣ Display the result ----------
print("Query:", query_sentence)
print("\nMost similar sentence found:")
print(results["matches"][0]["metadata"]["sentence"])
print(f"Similarity Score: {results['matches'][0]['score']:.4f}")


Query: AI is changing how companies work.

Most similar sentence found:
Artificial intelligence is transforming industries.
Similarity Score: 0.7986
